In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser" #username variable
password = "password" #password variable
shelter = AnimalShelter(username, password) #give .py username and password


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})) #get records from database



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()) 

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H2('Cody Poley'))), #name at top
    html.Center(html.Img(src='data:/home/codypoley_snhu/png;base64,{}'.format(encoded_image.decode()))), #picture location
    html.Center(html.A("Link to www.snhu.edu", href='https://www.snhu.edu', target="_blank")), #link to snhu.com
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))), #title
    html.Hr(),
    html.Div(dcc.RadioItems( #radio buttons for filter 
                options=[
                    {'label':'Water Rescue', 'value':'A'},
                    {'label':'Mountain or Wilderness Rescue', 'value':'B'},
                    {'label':'Disaster or Individual Tracking', 'value':'C'},
                    {'label':'Reset', 'value':'D'}
                ],
        value='D', #default radio button selected
        id='radio-filter', #id of the radio buttons
    )),
    html.Hr(),
    dt.DataTable(
        id='datatable-id', #id for the data table
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = True, #allow table to be edited
        filter_action="native", #filter by column
        sort_action="native", #sort by column
        sort_mode="multi", #allow for multiple sorting
        column_selectable="multi", #allowing selection of multiple columns
        row_selectable=True, #allowing selection of multiple rows
        row_deletable=False, #don't allow the rows to be deleted
        selected_columns=[], #allow for seletion of columns
        selected_rows=[], #allow for selection of rows
        page_action="native", #paging 
        page_current= 0, #start on first page
        page_size= 10, #10 animals per page
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
   html.Div(className='row',
         style={'display' : 'flex'}, 
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('radio-filter', 'value')])
def on_click(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (value == 'A'):
        df = pd.DataFrame(list(shelter.read({"$and": [ #filter for water rescue
            { "$or": 
             [{"breed":"Labrador Retriever Mix"},{"breed":"Chesapeake Bay Retriever"},{"breed":"Newfoundland"}]},
            {"sex_upon_outcome":"Intact Female"},
            {"$and":
             [{"age_upon_outcome_in_weeks":{"$gte":26}},{"age_upon_outcome_in_weeks":{"$lte":156}}]}]})))
       
    elif(value == 'B'):
        df = pd.DataFrame(list(shelter.read({"$and": [ #filter for mountain or wilderness rescue
            { "$or": 
             [{"breed":"German Sheperd"},{"breed":"Alaskan Malamute"},{"breed":"Old English Sheepdog"},{"breed":"Siberian Husky"}, {"breed":"Rottweiler"}]},
            {"sex_upon_outcome":"Intact Male"},
            {"$and":
             [{"age_upon_outcome_in_weeks":{"$gte":26}},{"age_upon_outcome_in_weeks":{"$lte":156}}]}]})))
       
    elif(value == 'C'):
        df = pd.DataFrame(list(shelter.read({"$and": [ #filter for disaster or individual tracking
            { "$or": 
             [{"breed":"Doberman Pinscher"},{"breed":"German Sheperd"},{"breed":"Golden Retriever"},{"breed":"Bloodhound"}, {"breed":"Rottweiler"}]},
            {"sex_upon_outcome":"Intact Male"},
            {"$and":
             [{"age_upon_outcome_in_weeks":{"$gte":20}},{"age_upon_outcome_in_weeks":{"$lte":300}}]}]})))
       
    elif(value == 'D'): #filter for reset
        df = pd.DataFrame.from_records(shelter.read({}))
        
        
        
        
    return df.to_dict('records')


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
     #add code for chart of your choice (e.g. pie chart) #
        dfPie = pd.DataFrame(list(viewData)) #get data from database
        return [
            dcc.Graph(          
                figure = px.pie(dfPie, names='breed') #create pie chart using value of breed
            )
        ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [ #create map size 
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[ #mark position of animal
                dl.Tooltip(dff.iloc[0,4]), 
                dl.Popup([
                    html.H1("Animal Name"), #animal name
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app